In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Load the dataset
df = pd.read_csv('data/credit.csv')

# Data preparation
X = df.drop(columns=['Class'])
y = df['Class']

# Normalize features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Convert to tensors
X_tensor = torch.tensor(X_normalized, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.long)

# Split into training and test datasets
X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor = train_test_split(
    X_tensor, y_tensor, test_size=0.2, random_state=42, stratify=y_tensor)

In [3]:
# Define a simple model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(X_train_tensor.shape[1], 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, criterion, optimizer, X_train, y_train, epochs=5):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 1 == 0:
            print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

In [4]:
# Define FGSM Attack
def fgsm_attack(model, data, target, epsilon):
    data.requires_grad = True
    output = model(data)
    loss = nn.CrossEntropyLoss()(output, target)
    model.zero_grad()
    loss.backward()
    data_grad = data.grad.data
    perturbed_data = data + epsilon * data_grad.sign()
    return perturbed_data

In [5]:
# Generate adversarial samples for training
epsilon = 0.1
model.eval()
adv_train_samples_tensor = fgsm_attack(model, X_train_tensor, y_train_tensor, epsilon)

# Generate adversarial samples for testing
adv_test_samples_tensor = fgsm_attack(model, X_test_tensor, y_test_tensor, epsilon)

# Convert adversarial samples to DataFrames and save
adv_train_samples_df = pd.DataFrame(adv_train_samples_tensor.detach().numpy(), columns=X.columns)
adv_test_samples_df = pd.DataFrame(adv_test_samples_tensor.detach().numpy(), columns=X.columns)

adv_train_samples_df['Class'] = y_train_tensor.numpy()
adv_test_samples_df['Class'] = y_test_tensor.numpy()

adv_train_samples_df.to_csv('adversarial_train_samples.csv', index=False)
adv_test_samples_df.to_csv('adversarial_test_samples.csv', index=False)

print("\nAdversarial training and testing samples saved successfully.")


Adversarial training and testing samples saved successfully.
